# Notebook Title

Brief Explanation

## If necessary install pytorch lightning

In [27]:
# pip install pytorch-lightning==0.9.0rc2

## If you want to use TPU install xla

In [28]:
# VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

## Imports

In [29]:
import torch
from torch import nn
from torch.optim import lr_scheduler

import pytorch_lightning as pl

import torchvision
import torchvision.models as models
from torchvision import transforms

from torch.utils.data import DataLoader
from torch.nn import functional as F
from pytorch_lightning.metrics import Accuracy, Recall, Precision, ROC, AUC

from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np

## Configs

In [30]:
TENSORBOARD_DIRECTORY = "logs/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Lightning DataModule

In [39]:
class FashionMNISTDataModule(pl.LightningDataModule):
    
    def __init__(self, root='./data/FashionMNIST'):
        self.root = root
        self.batch_size = 1024
        
        super().__init__()

    def setup(self, stage=None):
        self.trainset = torchvision.datasets.FashionMNIST(
            root = self.root,
            train = True,
            download = True,
            transform = transforms.Compose([
                transforms.ToTensor()]))

        self.valset = torchvision.datasets.FashionMNIST(
            root = self.root,
            train = False,
            download = True,
            transform = transforms.Compose([
                transforms.ToTensor()]))

    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=self.batch_size, num_workers=4, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.valset, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size)


## Create Pytorch Ligtning Model

In [32]:
class FashionMnistClassifer(pl.LightningModule):
    def __init__(self):
        super(FashionMnistClassifer, self).__init__()
        
        self.layer_1 = nn.Linear(28*28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)        
        
        
        self.criterion = F.nll_loss
        self.metrics = {"accuracy": Accuracy(), "recall": Recall()}
    
    def forward(self, x):
        batch_size, channels, width, height = x.size()
        
        x = x.view(batch_size, -1)
        
        x = self.layer_1(x)
        x = torch.relu(x)
        
        x = self.layer_2(x)
        x = torch.relu(x)
        
        x = self.layer_3(x)
        
        return torch.log_softmax(x, dim=1)
    
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

        scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
        
        return [optimizer], [scheduler]
    
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        
        loss = self.criterion(logits, y)

        tensorboard_logs = {'train_loss': loss}
        
        return {'loss': loss, 'log': tensorboard_logs}

    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        
        loss = self.criterion(logits, y)
        
        metrics_dict = {f"val_{name}": metric(logits, y) for name, metric in self.metrics.items()}
        
        return {**{"val_loss": loss}, **metrics_dict}
        
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()

        tensorboard_logs = {name: torch.stack([x[f"val_{name}"] for x in outputs]).mean()
                                for name, metric in self.metrics.items()}
        
        tensorboard_logs["val_loss"] = avg_loss

        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

## Lightning Trainer

In [33]:
logger = TensorBoardLogger(TENSORBOARD_DIRECTORY, name="logger_name")

In [40]:
model = FashionMnistClassifer()

dm = FashionMNISTDataModule()

In [41]:
trainer = pl.Trainer(max_epochs=10,
                     logger=logger,
                     gpus=0,
                     auto_scale_batch_size='power',
                     early_stop_callback=True)

trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


MisconfigurationException: Field batch_size not found in both `model` and `model.hparams`

## Add visualizations to Tensorboard

In [ ]:
writer = SummaryWriter(TENSORBOARD_DIRECTORY)

valloader = model.val_dataloader()
inputs, labels = next(iter(valloader))
inputs, labels = inputs.to(device), labels.to(device)


In [ ]:
grid = torchvision.utils.make_grid(inputs[:25])
writer.add_image('images', grid, 0)

writer.add_graph(model, inputs)
writer.close()

In [ ]:
%load_ext tensorboard

In [ ]:
# %reload_ext tensorboard
%tensorboard --logdir logs

## Show model metrics

In [ ]:
model.eval()

final_metrics = {}

for name, metric in model.metrics.items():
        final_metrics[f"{name}"]= torch.stack([metric(model(x), y) for x, y in valloader]).mean()
        
final_metrics

## Show Example Predictions

In [ ]:
outputs = model(inputs)
predictions = torch.max(outputs, 1)[1]

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress','Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
plt.figure(figsize=(20,20))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(inputs[i].view(28,28), cmap=plt.cm.binary)
    plt.xlabel(f"Real: {class_names[labels[i]]}, Pred: {class_names[predictions[i]]}")
plt.show()